In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

import torch as pt
import torch.optim as optim
import imageio
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [ ]:
#SinGAN

class ConvBlock(nn.Sequential):
    def __init__(self, in_channel, out_channel, ker_size, padd, stride):
        super(ConvBlock,self).__init__()
        self.add_module('conv',nn.Conv2d(in_channel ,out_channel,kernel_size=ker_size,stride=stride,padding=padd)),
        self.add_module('norm',nn.BatchNorm2d(out_channel)),
        self.add_module('LeakyRelu',nn.LeakyReLU(0.2, inplace=True))


class WDiscriminator(nn.Module):
    def __init__(self, opt):
        super(WDiscriminator, self).__init__()
        self.is_cuda = torch.cuda.is_available()
        N = int(opt.nfc)
        self.head = ConvBlock(opt.nc_im,N,opt.ker_size,opt.padd_size,1)
        self.body = nn.Sequential()
        for i in range(opt.num_layer-2):
            N = int(opt.nfc/pow(2,(i+1)))
            block = ConvBlock(max(2*N,opt.min_nfc),max(N,opt.min_nfc),opt.ker_size,opt.padd_size,1)
            self.body.add_module('block%d'%(i+1),block)
        self.tail = nn.Conv2d(max(N,opt.min_nfc),1,kernel_size=opt.ker_size,stride=1,padding=opt.padd_size)

    def forward(self, x, is_loss=False):
        x1 = self.head(x)
        x2 = self.body(x1)
        x = self.tail(x2)

        if is_loss:
            return [x1, x2, x]

        return x

In [ ]:
class MDFLoss(nn.Module):
    def __init__(self, saved_ds_path, cuda_available = True):
        super(MDFLoss, self).__init__()

        if cuda_available:
            self.Ds = torch.load(saved_ds_path)
        else:
            self.Ds = torch.load(saved_ds_path, map_location=torch.device('cpu'))

        self.num_discs = len(self.Ds)

    def forward(self, x, y, num_scales=8, is_ascending=1):
        # Get batch_size
        batch_size = x.shape[0]

        # Initialize loss vector
        loss = torch.zeros([batch_size]).to(x.device)
        # For every scale
        for scale_idx in range(num_scales):
            # Reverse if required
            if is_ascending:
                scale = scale_idx
            else:
                scale = self.num_discs - 1 - scale_idx

            # Choose discriminator
            D = self.Ds[scale]

            # Get discriminator activations
            pxs = D(x, is_loss=True)
            pys = D(y, is_loss=True)

            # For every layer in the output
            for idx in range(len(pxs)):
                # Compute L2 between representations
                l2 = (pxs[idx] - pys[idx])**2
                l2 = torch.mean(l2, dim=(1, 2, 3))

                # Add current difference to the loss
                loss += l2

        # Mean loss
        loss = torch.mean(loss)

        return loss

In [ ]:
%cd drive/MyDrive/ColabNotebooks/MDF-loss

/content/drive/MyDrive/ColabNotebooks/MDF-loss


In [ ]:
# Set parameters
cuda_available = False
#epochs = 25
epochs = 40
application = 'SISR'
image_path = './misc/i10.png'

if application =='SISR':
    path_disc = "./weights/Ds_SISR.pth"
elif application == 'Denoising':
    path_disc = "./weights/Ds_Denoising.pth"
elif application == 'JPEG':
    path_disc = "./weights/Ds_JPEG.pth"

# Read reference images
imgr = imageio.imread(image_path)
imgr = pt.from_numpy(imageio.core.asarray(imgr/255.0))
imgr = imgr.type(dtype=pt.float64)
imgr = imgr.permute(2,0,1)
imgr = imgr.unsqueeze(0).type(pt.FloatTensor)

# Create a noisy image
imgd = pt.rand(imgr.size())

# Save the original state
imgdo = imgd.detach().clone()

if cuda_available:
    imgr = imgr.cuda()
    imgd = imgd.cuda()

# Convert images to variables to support gradients
imgrb = Variable( imgr, requires_grad = False)
imgdb = Variable( imgd, requires_grad = True)

optimizer = optim.Adam([imgdb], lr=0.1)

# Initialise the loss
criterion = MDFLoss(path_disc, cuda_available=cuda_available)

# Iterate over the epochs optimizing for the noisy image
for ii in range(0,epochs):

    optimizer.zero_grad()
    loss = criterion(imgrb,imgdb)
    print("Epoch: ",ii+1," loss: ", loss.item())
    loss.backward()
    optimizer.step()



Epoch:  0  loss:  46.82178497314453
Epoch:  1  loss:  22.32317352294922
Epoch:  2  loss:  12.335370063781738
Epoch:  3  loss:  9.212684631347656
Epoch:  4  loss:  8.49681568145752
Epoch:  5  loss:  8.19428539276123
Epoch:  6  loss:  7.828133583068848
Epoch:  7  loss:  7.450799465179443
Epoch:  8  loss:  7.038461208343506
Epoch:  9  loss:  6.541945934295654
Epoch:  10  loss:  5.992953777313232
Epoch:  11  loss:  5.440065383911133
Epoch:  12  loss:  4.9054436683654785
Epoch:  13  loss:  4.421304702758789
Epoch:  14  loss:  4.0077924728393555
Epoch:  15  loss:  3.636884927749634
Epoch:  16  loss:  3.2704877853393555
Epoch:  17  loss:  2.9180796146392822
Epoch:  18  loss:  2.609934091567993
Epoch:  19  loss:  2.352980136871338
Epoch:  20  loss:  2.1337950229644775
Epoch:  21  loss:  1.9382336139678955
Epoch:  22  loss:  1.7594687938690186
Epoch:  23  loss:  1.5986530780792236
Epoch:  24  loss:  1.457843542098999
Epoch:  25  loss:  1.3348051309585571
Epoch:  26  loss:  1.224718451499939
Epo

In [ ]:
# Convert images to numpy
imgrnp = imgr.cpu().squeeze(0).permute(1,2,0).data.numpy()
imgdnp = imgd.cpu().squeeze(0).permute(1,2,0).data.numpy()
imgdonp = imgdo.cpu().squeeze(0).permute(1,2,0).data.numpy()


# Plot optimization results
fig, axs = plt.subplots(1, 3,figsize=(45,15))


axs[0].imshow(imgdonp)
axs[0].set_title('Noisy image',fontsize=48)
axs[1].imshow(imgdnp)
axs[1].set_title('Recovered image',fontsize=48)
axs[2].imshow(imgrnp)
axs[2].set_title('Reference image',fontsize=48)

# Remove the ticks from the axis
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

Output hidden; open in https://colab.research.google.com to view.